In [1]:
    import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import gc
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.ops.numpy_ops import np_utils
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from matplotlib import pyplot
#from keras.optimizers import Adam

In [2]:

# from tensorflow.keras.utils import to_categorical
# X1 = np.array(X)
# y1 = np.array(Y)
# #X = X.reshape(-1)
# #Y = to_categorical(Y, 2)
# print(len(y1), len(X1))

In [3]:
# X = np.array(X)
# Y = to_categorical(Y, 2)
# X = X.reshape(-1, 128, 128, 3)


In [4]:
# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
# X = X.reshape(-1,1,1,1)
# print(len(X_train), len(Y_train))
# print(len(X_val), len(Y_val))

In [5]:
image_size = (128, 128)
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0
X = [] # ELA converted images
Y = [] # 0 for fake, 1 for real

In [6]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [7]:
image_size = (128, 128)

In [8]:
df = pd.DataFrame(columns=['Image', 'Label'])

In [9]:
import random
path = '/kaggle/input/casia-dataset/CASIA2/Au/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(filename)
            Y.append(1)
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

random.shuffle(X)
X = X[:2100]
Y = Y[:2100]
print(len(X), len(Y))
# print(X)



In [10]:
path = '/kaggle/input/casia-dataset/CASIA2/Tp/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(filename)
            Y.append('0')
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

print(len(X), len(Y))
print(Y)

In [11]:
X = np.array(X)
Y = np.array(Y)

In [12]:
# X = np.array(X)
#Y = to_categorical(Y,2)
#X = X.reshape(-1, 128, 128, 3)

In [13]:
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 1
N_SPLIT = 10

In [14]:
# Storing the average of all predictions
import pandas as pd
main_pred = []
error = []
data_kfold = pd.DataFrame()

In [15]:
df = pd.DataFrame(columns=['Image', 'Label'])
df['Image'] = X.tolist()
df['Label'] = Y.tolist()
df.head()

In [16]:
df["Label"] = df["Label"].astype(str).astype(int)
train_y = df['Label']
train_x = df.drop(['Label'],axis=1)
print(train_y.shape)


In [17]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [18]:
from PIL import Image

%matplotlib inline
from tensorflow import keras 
from tensorflow.keras.applications import ResNet50,ResNet101
import cv2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [19]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [20]:

import glob
import shutil
import os
# os.remove('/kaggle/working/merge_folder')
# shutil.os.mkdir('/kaggle/working/' + 'merge_folder1')
# src_dir1 = '/kaggle/input/casia-dataset/CASIA2/Tp'
# src_dir2 = '/kaggle/input/casia-dataset/CASIA2/Au'
# dst_dir = '/kaggle/working/merge_folder'
# for jpgfile in glob.iglob(os.path.join(src_dir1, "*",  "*.jpg")):
#     shutil.copy(jpgfile, dst_dir)
    
# for jpgfile in glob.iglob(os.path.join(src_dir2, "*", "*.jpg")):
#     shutil.copy(jpgfile, dst_dir)
    
# print("/kaggle/working/merge_folder")

from distutils.dir_util import copy_tree
src_dir1 = '/kaggle/input/casia-dataset/CASIA2/Tp'
src_dir2 = '/kaggle/input/casia-dataset/CASIA2/Au'
dst_dir = '/kaggle/working/merge_folder'
copy_tree(src_dir1,dst_dir)
copy_tree(src_dir2,dst_dir)

In [21]:
TRAIN_PATH ='/kaggle/working/merge_folder'

In [22]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
#     
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    return model
#     model.summary()


In [23]:
df['Label'].value_counts()

In [24]:
from sklearn.utils.multiclass import type_of_target
type_of_target(Y)



In [25]:
print(train_x.shape)
print(train_y.shape)
print(df.Label.shape)

In [26]:
# import tensorflow as tf
# train_y = np.array(df['Label'].tolist()) 
# train_y = tf.one_hot(train_y, depth=2)

In [27]:

IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 1
N_SPLIT = 10
#Initializing Data Generators
train_datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

# k-fold
kfold = StratifiedKFold(n_splits=N_SPLIT,shuffle=True,random_state=42)

# Variable for keeping count of split we are executing
j = 0

# K-fold Train and test for each split
for train_idx, val_idx in list(kfold.split(train_x,train_y)):
    x_train_df = df.iloc[train_idx]
    x_valid_df = df.iloc[val_idx]
    j+=1


#     training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
#                                                  x_col="Image", y_col="Label",
#                                                  class_mode="raw",
#                                                  target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
#     validation_set = validation_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
#                                                  x_col="Image", y_col="Label",
#                                                  class_mode="raw",
#                                                  target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
    training_set = train_datagen.flow_from_directory( '/kaggle/input/casia-dataset/CASIA2', target_size=(128, 128), color_mode='rgb', classes=None,
                                                     class_mode='categorical',  shuffle=True, seed=None,interpolation='nearest')
    validation_set = validation_datagen.flow_from_directory( '/kaggle/input/casia-dataset/CASIA2', target_size=(128, 128), color_mode='rgb', classes=None,
                                                     class_mode='categorical',  shuffle=True, seed=None,interpolation='nearest')
    
    model_test = build_model()
   
    hist = model_test.fit( training_set,
                                        validation_data=validation_set,
                                        epochs = EPOCHS,
                                        steps_per_epoch=12
#                           x_train_df.shape[0] // BATCH_SIZE
                                        )
    predictions = model_test.predict(validation_set, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_set.class_indices
    ytrue = validation_set.classes
#     test_generator = ImageDataGenerator(rescale = 1./255)
    
#     test_set = test_generator.flow_from_dataframe(dataframe=df, directory=TRAIN_PATH,
#                                                  x_col="Image",y_col=None,
#                                                  class_mode=None,
#                                                  target_size=(IMG_SIZE,IMG_SIZE))
    
#     pred= model_test.predict_generator(test_set, len(df) // BATCH_SIZE)
#     predicted_class_indices=np.argmax(pred,axis=1)
                                       
    data_kfold[j] = true_classes
    gc.collect()


In [28]:

print(predictions, true_classes, ytrue)

In [29]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score
print("***Performance on Validation data***")    
valAcc, valPrec, valFScore = my_metrics(ytrue, yPredictions)

In [30]:
labels=(training_set.class_indices)
labels2=dict((v,k) for k,v in labels.items())
import collections 
for i in range(len(data_kfold)):
    co = collections.Counter(data_kfold.loc[i])
    co = sorted(co.items(),key=lambda x: x[1],reverse=True)
    ans.Class.loc[i] = labels2[co[0][0]]

In [ ]:
predictions = model_test.predict_generator(validation_set, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = validation_set.class_indices

In [ ]:
data_kfold

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    


In [ ]:
# Predict the values from the validation dataset
Y_pred = model_test.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))

In [ ]:
class_names = ['fake', 'real']

In [ ]:
real_image_path = '/kaggle/input/casia-dataset/casia/CASIA2/Au/Au_ani_00001.jpg'
image = prepare_image(real_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model_test.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

fake_image_path = '/kaggle/input/casia-dataset/casia/CASIA2/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg'
image = prepare_image(fake_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model_test.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')


In [ ]:
fake_image = os.listdir('/kaggle/input/casia-dataset/casia/CASIA2/Tp/')
correct = 0
total = 0
for file_name in fake_image:
    if file_name.endswith('jpg') or filename.endswith('png'):
        fake_image_path = os.path.join('/kaggle/input/casia-dataset/casia/CASIA2/Tp/', file_name)
        image = prepare_image(fake_image_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model_test.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        total += 1
        if y_pred_class == 0:
            correct += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')
